In [ ]:
import string
import types
import pandas as pd
import nltk
import numpy as np
from numpy.random import randn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.metrics import classification_report
import time
from sklearn import svm
import codecs, csv, sys
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB                                                                
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score
import itertools
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV

In [ ]:
# loading the csv containg embedding for test and train embedding data
df_train = pd.read_csv("/Volumes/Barly/NLP/Project/5k_testing/Type1/data/Word2Vec/train_word2vec.csv")
df_test = pd.read_csv("/Volumes/Barly/NLP/Project/5k_testing/Type1/data/Word2Vec/test_word2vec.csv")

In [ ]:
# Concatenating the embedding( sum and average) for question1 and question 2 and normalizing before passing it to classifiers
start_time = time.time()

train_embedding1_sum = map(np.fromstring, df_train['embedding_1_sum'],\
                                      itertools.repeat(float, df_train.shape[0]),\
                                                       itertools.repeat(300, df_train.shape[0]))

train_embedding2_sum = map(np.fromstring, df_train['embedding_2_sum'],\
                                      itertools.repeat(float, df_train.shape[0]),\
                                                       itertools.repeat(300, df_train.shape[0]))

train_embedding1_avg = map(np.fromstring, df_train['embedding_1_avg'],\
                                      itertools.repeat(float, df_train.shape[0]),\
                                                       itertools.repeat(300, df_train.shape[0]))

train_embedding2_avg = map(np.fromstring, df_train['embedding_2_avg'],\
                                      itertools.repeat(float, df_train.shape[0]),\
                                                       itertools.repeat(300, df_train.shape[0]))
X_label = np.array(df_train['is_duplicate'].tolist())
X_sum = np.concatenate((np.array(train_embedding1_sum), np.array(train_embedding2_sum)), axis=1)
normalize_sum = Normalizer().fit(X_sum)
X_sum = normalize_sum.transform(X_sum)

X_avg = np.concatenate((np.array(train_embedding1_avg), np.array(train_embedding2_avg)), axis=1)
normalize_avg = Normalizer().fit(X_avg)
X_avg = normalize_avg.transform(X_avg)

test_embedding1_sum = map(np.fromstring, df_test['embedding_1_sum'],\
                                      itertools.repeat(float, df_test.shape[0]),\
                                                       itertools.repeat(300, df_test.shape[0]))

test_embedding2_sum = map(np.fromstring, df_test['embedding_2_sum'],\
                                      itertools.repeat(float, df_test.shape[0]),\
                                                       itertools.repeat(300, df_test.shape[0]))

test_embedding1_avg = map(np.fromstring, df_test['embedding_1_avg'],\
                                      itertools.repeat(float, df_test.shape[0]),\
                                                       itertools.repeat(300, df_test.shape[0]))

test_embedding2_avg = map(np.fromstring, df_test['embedding_2_avg'],\
                                      itertools.repeat(float, df_test.shape[0]),\
                                                       itertools.repeat(300, df_test.shape[0]))
Y_label = np.array(df_test['is_duplicate'].tolist())
Y_sum = np.concatenate((np.array(test_embedding1_sum), np.array(test_embedding2_sum)), axis=1) # concatenating the embedding for question 1 and question2
Y_sum = normalize_sum.transform(Y_sum)

Y_avg = np.concatenate((np.array(test_embedding1_sum), np.array(test_embedding2_sum)), axis=1)
Y_avg = normalize_avg.transform(Y_avg)

In [ ]:
# Predicting is_duplicate value by passing embedding into shallow classifiers
def naive_bayes(x_train_matrix, x_test_matrix, y_train, y_test):    #NAIVE BAYERS MODEL                                                                
    nb = MultinomialNB(alpha=2.8)
    nb.fit(x_train_matrix, y_train)                                 #train the model
    y_pred = nb.predict(x_test_matrix)                              #make predictions for X_test
    print ("Naive Bayes Model score: " + str(accuracy_score(y_test, y_pred)))         
    print ("Naive Bayes Model confusion matrix:")
    print (confusion_matrix(y_test, y_pred))
    y_predicted_proba = nb.predict_proba(x_test_matrix)
    print log_loss(y_test, y_predicted_proba)
    return y_pred

def logistic_regression(x_train_matrix, x_test_matrix, y_train, y_test): 
    logReg = LogisticRegressionCV(Cs=[0.01, 0.1, 1, 10, 100], cv=5, solver='saga', n_jobs= -1 )
    logReg.fit(x_train_matrix, y_train)
    y_pred_log = logReg.predict(x_test_matrix)
    print ("Logistic Regression score: "+ str(accuracy_score(y_test, y_pred_log)))
    print ("Logistic Regression confusion matrix:")
    print (confusion_matrix(y_test, y_pred_log))
    y_predicted_proba = logReg.predict_proba(x_test_matrix)
    print log_loss(y_test, y_predicted_proba)
    return y_pred_log

def linear_svm(x_train_matrix, x_test_matrix, y_train, y_test):
    
    param_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']}]
    svc = svm.SVC()
    svm1 = GridSearchCV(estimator=svc, param_grid=param_grid, n_jobs= -1, cv = 5) 
    #svm1 = LinearSVC(C=1)
    svm1.fit(x_train_matrix, y_train)
    y_pred_svc = svm1.predict(x_test_matrix)
    print ("SVC score: " + str(accuracy_score(y_test, y_pred_svc)))
    print ("SVC confusionmatrix:")
    print (confusion_matrix(y_test, y_pred_svc))
    return y_pred_svc

In [ ]:
preprocessing_time = time.time()
print("--- start -> preprocessing:%s seconds ---" % (preprocessing_time - start_time))


print ("\n\n Naive bayes result on word_embedding_sum: ")
y_pred_nb_sum = naive_bayes(X_sum, Y_sum, X_label, Y_label)
print ("\n\n Naive bayes result on word_embedding_avg: ")
y_pred_nb_avg = naive_bayes(X_avg, Y_avg, X_label, Y_label)
naive_bayes_time = time.time() 
print("--- preprocessing -> naive bayes :%s seconds ---" % (naive_bayes_time - preprocessing_time))


print ("-------------------------------------------")
print ("\n\n Logistic Regression result on word_embedding_sum: ")
y_pred_lr_sum = logistic_regression(X_sum, Y_sum, X_label, Y_label)
print ("\n\n Logistic Regression result on word_embedding_avg: ")
y_pred_lr_avg = logistic_regression(X_avg, Y_avg, X_label, Y_label)
logistic_regression_time = time.time()
print("--- naive bayes -> logistic regression :%s seconds ---" % (logistic_regression_time - naive_bayes_time))

print ("-------------------------------------------")
print ("\n\n Linear SVM result on word_embedding_sum: ")
y_pred_svm_sum = linear_svm(X_sum, Y_sum, X_label, Y_label)
print ("\n\n Linear SVM result on word_embedding_avg: ")
y_pred_svm_avg = linear_svm(X_avg, Y_avg, X_label, Y_label)
linear_svm_time = time.time()
print("--- logistic regression -> linear svm :%s seconds ---" % (linear_svm_time - logistic_regression_time))
print("--- TOTAL TIME: %s seconds ---" % (time.time() - start_time))

In [ ]:
y_pred_nb_sum = y_pred_nb_sum.tolist()
y_pred_nb_avg = y_pred_nb_avg.tolist()
y_pred_lr_sum = y_pred_lr_sum.tolist()
y_pred_lr_avg = y_pred_lr_avg.tolist()
y_pred_svm_sum = y_pred_svm_sum.tolist()
y_pred_svm_avg = y_pred_svm_avg.tolist()

In [ ]:
df_test['nb_sum_duplicate'] = y_pred_nb_sum
df_test['nb_avg_duplicate'] = y_pred_nb_avg
df_test['lr_sum_duplicate'] = y_pred_lr_sum
df_test['lr_avg_duplicate'] = y_pred_lr_avg
df_test['svm_sum_duplicate'] = y_pred_svm_sum
df_test['svm_avg_duplicate'] = y_pred_svm_avg

In [ ]:
df_test.to_csv("/Volumes/Barly/NLP/Project/5k_testing/Type1/Results/test_word2vec_classifier.csv", index = False)

In [ ]:
#Comparing the cosine similarity between word2vec vectors on different thresholds
df_new = pd.read_csv("/Volumes/Barly/NLP/Project/5k_testing/Type1/Results/test_word2vec_classifier.csv")

def conv(elem):
    a = re.compile(r"[-+]?\d*\.\d+|\d+")
    return re.findall(a, elem)[0]

threshold_list = [0.6, 0.7,0.78, 0.8]
df_cos_test = df_new
df_cos_test['cos_avg'] = map(conv, df_cos_test['cos_avg'])
df_cos_test['cos_sum'] = map(conv, df_cos_test['cos_sum'])
df_cos_test[['cos_avg','cos_sum']] = df_cos_test[['cos_avg','cos_sum']].apply(pd.to_numeric)

for threshold in threshold_list:
    
    print "Threshold: ", threshold
    
    df_cos_test['thre_sum_'+str(threshold)] = np.where(df_cos_test['cos_sum']>=threshold, 1, 0)
    df_cos_test['thre_avg_'+str(threshold)] = np.where(df_cos_test['cos_avg']>=threshold, 1, 0)

    print confusion_matrix(df_cos_test['is_duplicate'], df_cos_test['thre_sum_'+str(threshold)]\
                                      , labels=target_names).ravel()
    print classification_report(df_cos_test['is_duplicate'], df_cos_test['thre_sum_'+str(threshold)])
    print accuracy_score(df_cos_test['is_duplicate'], df_cos_test['thre_sum_'+str(threshold)])
    print confusion_matrix(df_cos_test['is_duplicate'], df_cos_test['thre_avg_'+str(threshold)]\
                                      , labels=target_names).ravel()
    
    print classification_report(df_cos_test['is_duplicate'], df_cos_test['thre_avg_'+str(threshold)])

    print "Accuracy: " ,accuracy_score(df_cos_test['is_duplicate'], df_cos_test['thre_avg_'+str(threshold)])
    print "\n*******************"
